<h1>BÀI TẬP LỚN Môn : Xử Lý Ngôn Ngữ Tự Nhiên</h1>

-----------------------

*Nhóm 21 - Chủ đề : Tóm tắt văn bản (Text Summarization)*
-----------------------
* Nguyễn Việt Anh - 2121050054
* Phạm Thị Liên - 212151542
* Lê Danh Cần - 2121051529

<h2>Extractive Summarization</h2>
<h3>Sentence Scoring</h3>

In [142]:
import nltk
from underthesea import word_tokenize, sent_tokenize
import re
import numpy as np 

# with open('New folder/output_2024-05-27_21-18-47.txt', 'r', encoding='utf-8') as file:
#     text = file.read()
text = """ Tổng quan khóa học
Khóa học này sẽ dẫn dắt bạn đi từ cơ bản đến tạo ra một sản phẩm hoàn chỉnh viết bằng C++.
Sẽ giúp người học hiểu được cách xây dựng một phần mềm từ đầu đến cuối.
Nội dung giảng dạy sẽ bao gồm kiến thức về ngôn ngữ, trắc nghiệm để kiểm tra kiến thức, bài tập thực hành nộp cho TA (Teaching assistant) để họ chấm.
Khi thắc mắc về nội dung bài học thì người học có thể hỏi trực tiếp Đạt hoặc các bạn TA.
Khi học xong sẽ được cấp chứng nhận đã học xong khóa học.
 """

with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

# Hàm xử lý văn bản
def text_handle(text):
    lower_text = text.lower()
    clean_text = re.sub(r'\s+', ' ', lower_text)
    return clean_text.strip()

# Hàm loại bỏ từ dừng
def remove_stopwords(sentence, stopwords):
    words = word_tokenize(sentence)
    filtered_sentence = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_sentence)

# Hàm loại bỏ dấu câu 
def remove_punctuation(sentence):
    sentence = re.sub(r'\d+', '', sentence)
    return re.sub(r'[^\w\s]','',sentence)

# Tách các câu trong văn bản
sentences = sent_tokenize(text_handle(text))

# Xử lý từng câu
processed_sentences = []
for sentence in sentences: 
    cleaned_sentence = remove_stopwords(sentence, stopwords)
    cleaned_sentence = remove_punctuation(cleaned_sentence)
    processed_sentences.append(cleaned_sentence)


print("Các câu sau khi xử lý:")
print(processed_sentences)


Các câu sau khi xử lý:
['tổng quan khóa học khóa học dẫn dắt đi cơ bản sản phẩm hoàn chỉnh viết c   ', 'giúp học xây dựng phần mềm đầu ', 'nội dung giảng dạy bao gồm kiến thức ngôn ngữ  trắc nghiệm kiểm tra kiến thức  bài tập thực hành nộp ta  teaching assistant  chấm ', 'thắc mắc nội dung bài học học ta ', 'học xong chứng nhận học xong khóa học ']


In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Hàm tính điểm TF-IDF cho từng câu
def calculate_tfidf(sentences):
    tfidf_vector = TfidfVectorizer()
    tfidf_matrix = tfidf_vector.fit_transform(sentences)
    tfidf_score = np.asarray(tfidf_matrix.sum(axis=1)).ravel()
    return tfidf_score

# Hàm tính điểm cho mỗi câu dựa trên chiều dài và điểm TF-IDF
def calculate_sentence_scores(sentences):
    tfidf = calculate_tfidf(sentences)
    print('Ma trận điểm TF-IDF: ',tfidf)
    sentence_lengths = [len(sentence.split()) for sentence in sentences]
    print('Ma trận độ dài từng câu: ',sentence_lengths)
    max_sentence_length = max(sentence_lengths)
    normalized_lengths = [length / max_sentence_length for length in sentence_lengths]
    print('Điểm chuẩn hóa câu trong văn bản: ',normalized_lengths)
    sentence_scores = [tfidf_score * length for tfidf_score, length in zip(tfidf, normalized_lengths)]
    return sentence_scores

sentence_scores = calculate_sentence_scores(processed_sentences)

# Hiển thị điểm của từng câu
print(sentence_scores)

Ma trận điểm TF-IDF:  [3.69977021 2.61131061 4.61125782 2.62182286 2.10705287]
Ma trận độ dài từng câu:  [17, 7, 25, 8, 8]
Điểm chuẩn hóa câu trong văn bản:  [0.68, 0.28, 1.0, 0.32, 0.32]
[2.515843742026007, 0.731166971481899, 4.611257815416505, 0.8389833144418211, 0.6742569184013338]


In [144]:
def summarize_text(sentences, sentence_scores, num_sentences):
    # Sắp xếp các câu theo điểm số giảm dần
    sorted_indices = sorted(range(len(sentence_scores)), key=lambda i: sentence_scores[i], reverse=True)
    # Chọn ra các câu có điểm cao nhất để tạo tóm tắt
    summary_indices = sorted_indices[:num_sentences]
    # Sắp xếp lại các chỉ số câu theo thứ tự xuất hiện trong văn bản
    summary_indices.sort()
    # Tạo tóm tắt bằng cách lấy các câu được chọn
    summary = [sentences[i] for i in summary_indices]
    return summary


num_summary_sentences = 1
summary = summarize_text(processed_sentences, sentence_scores, num_summary_sentences)


print("Văn bản tóm tắt:")
for i in summary: 
    print(text_handle(i))


Văn bản tóm tắt:
nội dung giảng dạy bao gồm kiến thức ngôn ngữ trắc nghiệm kiểm tra kiến thức bài tập thực hành nộp ta teaching assistant chấm
